In [12]:
# =========================
# CELL 1 — Imports + Load data + basic setup
# =========================
import numpy as np
import pandas as pd

DATA_PATH = "historical_2003-2014.csv"   # اسم فایل خودت
TARGET_COL = "cdm_historical"

# این ستون‌ها فعلاً نگه می‌داریم (بعداً برای mapping لازم میشن)
ID_COLS = ["point_id", "lat_dd", "long_dd"]

# ستون‌هایی که فعلاً در مدل استفاده نمی‌کنیم (ولی بعداً برای تحلیل/سناریوها نگه می‌داریم)
AUX_COLS = ["gdp", "pop", "cdm_model", "cdm_p_value"]

df = pd.read_csv(DATA_PATH)

print("Shape:", df.shape)
print("Target NaN:", df[TARGET_COL].isna().sum(), "/", len(df), f"({100*df[TARGET_COL].isna().mean():.2f}%)")

# فقط چک کنیم ستون‌های کلیدی موجودند
for c in ID_COLS + [TARGET_COL]:
    assert c in df.columns, f"Missing column: {c}"


Shape: (244016, 34)
Target NaN: 145534 / 244016 (59.64%)


In [13]:
# =========================
# CELL 2 — Create spatial folds (KMeans on coordinates) for LABELED data only
# =========================
from sklearn.cluster import KMeans

N_FOLDS = 5
SEED = 42

# فقط داده‌های برچسب‌دار برای ساخت فولد
df_labeled = df[df[TARGET_COL].notna()].copy()

coords = df_labeled[["lat_dd", "long_dd"]].to_numpy()

kmeans = KMeans(n_clusters=N_FOLDS, random_state=SEED, n_init=10)
df_labeled["fold_id"] = kmeans.fit_predict(coords)

print("Labeled shape:", df_labeled.shape)
print("Fold sizes:")
print(df_labeled["fold_id"].value_counts().sort_index())


Labeled shape: (98482, 35)
Fold sizes:
fold_id
0    19450
1    21194
2    13359
3    21629
4    22850
Name: count, dtype: int64


In [14]:
# =========================
# CELL 3 — Sanity checks: class distribution per fold (counts + %)
# =========================
fold_class_counts = (
    df_labeled.groupby("fold_id")[TARGET_COL]
    .value_counts()
    .unstack(fill_value=0)
    .sort_index()
)

print("\nClass distribution per fold (counts):")
display(fold_class_counts)

print("\nClass distribution per fold (row %):")
fold_class_pct = fold_class_counts.div(fold_class_counts.sum(axis=1), axis=0) * 100
display(fold_class_pct.round(2))

missing_classes_per_fold = (fold_class_counts == 0).sum(axis=1)
print("\n# of missing classes per fold:")
print(missing_classes_per_fold)



Class distribution per fold (counts):


cdm_historical,0.0,1.0,2.0,3.0
fold_id,,,,
0,10137,3935,4653,725
1,2186,7374,10017,1617
2,10860,2499,0,0
3,10790,4938,4885,1016
4,1698,6908,11100,3144



Class distribution per fold (row %):


cdm_historical,0.0,1.0,2.0,3.0
fold_id,,,,
0,52.12,20.23,23.92,3.73
1,10.31,34.79,47.26,7.63
2,81.29,18.71,0.00,0.00
3,49.89,22.83,22.59,4.70
4,7.43,30.23,48.58,13.76



# of missing classes per fold:
fold_id
0    0
1    0
2    2
3    0
4    0
dtype: int64


In [ ]:
# =========================
# CELL 4 — Enforced spatial fold repair (minimum samples per class)
# =========================
from scipy.spatial.distance import cdist

MIN_SAMPLES_PER_CLASS = 50  # می‌تونی 30، 50 یا 100 بگذاری

fold_centers = (
    df_labeled.groupby("fold_id")[["lat_dd", "long_dd"]]
    .mean()
)

for fold_id in fold_class_counts.index:
    for cls in fold_class_counts.columns:
        current_count = fold_class_counts.loc[fold_id, cls]
        
        if current_count >= MIN_SAMPLES_PER_CLASS:
            continue
        
        needed = MIN_SAMPLES_PER_CLASS - current_count
        print(f"Fold {fold_id}, class {cls}: adding {needed} samples")
        
        # candidates from other folds
        candidates = df_labeled[
            (df_labeled[TARGET_COL] == cls) &
            (df_labeled["fold_id"] != fold_id)
        ].copy()
        
        if candidates.empty:
            print(f"  ⚠️ No candidates available for class {cls}")
            continue
        
        center = fold_centers.loc[fold_id].values.reshape(1, -1)
        dists = cdist(candidates[["lat_dd", "long_dd"]], center)
        candidates["dist"] = dists
        
        to_move = candidates.sort_values("dist").head(needed).index
        df_labeled.loc[to_move, "fold_id"] = fold_id


Fold 2, class 2.0: adding 49 samples
Fold 2, class 3.0: adding 49 samples


In [18]:
# Re-check class distribution
fold_class_counts = (
    df_labeled.groupby("fold_id")[TARGET_COL]
    .value_counts()
    .unstack(fill_value=0)
    .sort_index()
)

display(fold_class_counts)
display((fold_class_counts.div(fold_class_counts.sum(axis=1), axis=0) * 100).round(2))


cdm_historical,0.0,1.0,2.0,3.0
fold_id,,,,
0,10137,3935,4603,675
1,2186,7374,10017,1617
2,10860,2499,50,50
3,10790,4938,4885,1016
4,1698,6908,11100,3144


cdm_historical,0.0,1.0,2.0,3.0
fold_id,,,,
0,52.39,20.34,23.79,3.49
1,10.31,34.79,47.26,7.63
2,80.69,18.57,0.37,0.37
3,49.89,22.83,22.59,4.70
4,7.43,30.23,48.58,13.76


In [ ]:
# =========================
# CELL 5 — Define feature groups (model-ready)
# =========================

TARGET_COL = "cdm_historical"

# ستون‌هایی که فعلاً در مدل استفاده نمی‌کنیم
NON_MODEL_COLS = [
    TARGET_COL,
    "fold_id",
    "point_id",
    "lat_dd",
    "long_dd",
    "gdp",
    "pop",
    "cdm_model",
    "cdm_p_value",
]

# categorical features (همونی که خودت درست تشخیص دادی)
CATEGORICAL_COLS = [
    "lulc",
    "lithology",
    "soil_texture",
    "soil_order",
]

# numerical features
NUMERICAL_COLS = [
    "aspect", "cdd", "dem",
    "dist_snowice", "dist_water",
    "evspsbl", "hurs", "lai",
    "mrro", "mrsos", "prcptot",
    "r10mm", "rlds", "rsds",
    "sfcWind", "slope",
    "tg_mean", "tx_max", "txgt_30",
    "dist_crop", "dist_roads", "dist_urban",
]

# sanity check
all_features = CATEGORICAL_COLS + NUMERICAL_COLS
missing = set(all_features) - set(df_labeled.columns)
assert len(missing) == 0, f"Missing features: {missing}"

print("Categorical features:", len(CATEGORICAL_COLS))
print("Numerical features:", len(NUMERICAL_COLS))
print("Total model features:", len(all_features))


Categorical features: 4
Numerical features: 22
Total model features: 26


In [25]:
# =========================
# BLOCK 6 — Fold-safe Preprocessor (Imputer + Scaling + OneHot)
# =========================
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

def build_preprocessor():
    num_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),     # ✅ NaN عددی
        ("scaler", StandardScaler())
    ])

    cat_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")), # ✅ NaN کتگوریکال
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", num_pipe, NUMERICAL_COLS),
            ("cat", cat_pipe, CATEGORICAL_COLS),
        ],
        remainder="drop",
        verbose_feature_names_out=False
    )
    return pre


def get_fold_split(df_labeled, fold_id):
    train_df = df_labeled[df_labeled["fold_id"] != fold_id].copy()
    val_df   = df_labeled[df_labeled["fold_id"] == fold_id].copy()
    return train_df, val_df


def fit_transform_fold(train_df, val_df):
    pre = build_preprocessor()

    Xtr_raw = train_df[NUMERICAL_COLS + CATEGORICAL_COLS]
    Xva_raw = val_df[NUMERICAL_COLS + CATEGORICAL_COLS]

    ytr = train_df[TARGET_COL].astype(int).to_numpy()
    yva = val_df[TARGET_COL].astype(int).to_numpy()

    Xtr = pre.fit_transform(Xtr_raw)
    Xva = pre.transform(Xva_raw)

    feat_names = pre.get_feature_names_out()
    return Xtr, ytr, Xva, yva, pre, feat_names


# sanity test روی fold=0
tr_df, va_df = get_fold_split(df_labeled, fold_id=0)
Xtr, ytr, Xva, yva, pre, feat_names = fit_transform_fold(tr_df, va_df)

print("Fold=0")
print("X_train:", Xtr.shape, "y_train:", ytr.shape)
print("X_val  :", Xva.shape, "y_val  :", yva.shape)
print("Total transformed features:", len(feat_names))
print("Any NaN in X_train?", np.isnan(Xtr).any())
print("Any NaN in X_val?", np.isnan(Xva).any())


Fold=0
X_train: (79132, 83) y_train: (79132,)
X_val  : (19350, 83) y_val  : (19350,)
Total transformed features: 83
Any NaN in X_train? False
Any NaN in X_val? False


In [31]:
# =========================
# CELL 7 — Plan B MRMR (MI - redundancy) fold-aware + stable aggregation
# =========================
import numpy as np
import pandas as pd
from sklearn.feature_selection import mutual_info_classif

K_MRMR = 15       # 10/15/20 
SEED = 42

RAW_ALL = NUMERICAL_COLS + CATEGORICAL_COLS

def _prepare_for_mi(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()

    # categorical -> codes
    for c in CATEGORICAL_COLS:
        if c in X.columns:
            X[c] = X[c].astype("category").cat.codes

    # impute (same as pipeline logic)
    for c in NUMERICAL_COLS:
        if c in X.columns:
            X[c] = X[c].fillna(X[c].median())
    for c in CATEGORICAL_COLS:
        if c in X.columns:
            mode = X[c].mode().iloc[0] if not X[c].mode().empty else -1
            X[c] = X[c].fillna(mode)

    return X

def mrmr_greedy_mi_redundancy(X: pd.DataFrame, y: np.ndarray, k: int = 20, seed: int = 42):
    """
    Greedy MRMR-like:
      relevance = MI(feature, y)
      redundancy = mean(abs(corr(feature, selected)))
      score = relevance - redundancy
    """
    Xp = _prepare_for_mi(X)

    # MI
    mi = mutual_info_classif(Xp.values, y, random_state=seed)
    mi = pd.Series(mi, index=Xp.columns)

    # Corr matrix (absolute) for redundancy
    corr = Xp.corr().abs().fillna(0.0)

    selected = []
    remaining = list(Xp.columns)

    for _ in range(min(k, len(remaining))):
        if len(selected) == 0:
            best = mi.loc[remaining].idxmax()
        else:
            red = corr.loc[remaining, selected].mean(axis=1)  # avg corr with selected
            score = mi.loc[remaining] - red
            best = score.idxmax()

        selected.append(best)
        remaining.remove(best)

    return selected, mi.sort_values(ascending=False)

# ---- fold-aware selection
fold_selected = {}
for fid in sorted(df_labeled["fold_id"].unique()):
    tr_df, va_df = get_fold_split(df_labeled, fold_id=fid)
    Xtr = tr_df[RAW_ALL]
    ytr = tr_df[TARGET_COL].astype(int).values

    sel, mi_rank = mrmr_greedy_mi_redundancy(Xtr, ytr, k=K_MRMR, seed=SEED)
    fold_selected[fid] = sel
    print(f"Fold {fid} selected top-{K_MRMR}:", sel)

# ---- aggregate stable ranking across folds (rank voting)
scores = {f: 0.0 for f in RAW_ALL}
for fid, sel in fold_selected.items():
    for r, f in enumerate(sel):
        scores[f] += (K_MRMR - r)  # higher if appears early

mrmr_rank = (
    pd.DataFrame({"feature": list(scores.keys()), "score": list(scores.values())})
    .sort_values("score", ascending=False)
    .reset_index(drop=True)
)

print("\nAggregated stable ranking (top 30):")
display(mrmr_rank.head(30))

K_FINAL = 20
SELECTED_RAW_FEATURES = mrmr_rank["feature"].head(K_FINAL).tolist()
print(f"\nFINAL SELECTED_RAW_FEATURES (K={K_FINAL}):")
print(SELECTED_RAW_FEATURES)


Fold 0 selected top-15: ['dist_snowice', 'evspsbl', 'prcptot', 'aspect', 'rsds', 'soil_texture', 'lai', 'soil_order', 'dist_roads', 'sfcWind', 'mrsos', 'mrro', 'lithology', 'dem', 'cdd']
Fold 1 selected top-15: ['dist_snowice', 'dist_roads', 'soil_order', 'lai', 'rsds', 'mrro', 'soil_texture', 'sfcWind', 'mrsos', 'aspect', 'hurs', 'lithology', 'evspsbl', 'cdd', 'dem']
Fold 2 selected top-15: ['dist_snowice', 'evspsbl', 'sfcWind', 'soil_order', 'dist_roads', 'soil_texture', 'dem', 'rsds', 'aspect', 'lithology', 'mrsos', 'mrro', 'lai', 'tg_mean', 'cdd']
Fold 3 selected top-15: ['dist_snowice', 'mrro', 'dist_roads', 'lai', 'soil_order', 'rsds', 'sfcWind', 'mrsos', 'soil_texture', 'aspect', 'evspsbl', 'dem', 'lithology', 'cdd', 'dist_crop']
Fold 4 selected top-15: ['dist_roads', 'dist_snowice', 'soil_texture', 'soil_order', 'evspsbl', 'lithology', 'lulc', 'dist_crop', 'mrro', 'lai', 'rsds', 'aspect', 'sfcWind', 'mrsos', 'hurs']

Aggregated stable ranking (top 30):


,feature,score
0,dist_snowice,74.0
1,dist_roads,60.0
2,soil_order,56.0
3,soil_texture,49.0
4,evspsbl,47.0
5,rsds,45.0
6,lai,42.0
7,mrro,39.0
8,sfcWind,39.0
9,aspect,35.0



FINAL SELECTED_RAW_FEATURES (K=20):
['dist_snowice', 'dist_roads', 'soil_order', 'soil_texture', 'evspsbl', 'rsds', 'lai', 'mrro', 'sfcWind', 'aspect', 'mrsos', 'lithology', 'dem', 'prcptot', 'lulc', 'dist_crop', 'hurs', 'cdd', 'tg_mean', 'tx_max']


In [32]:
# =========================
# CELL 8 — Preprocessor for SELECTED_RAW_FEATURES (subset-aware)
# =========================
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

def split_selected_features(selected_features):
    sel_num = [c for c in selected_features if c in NUMERICAL_COLS]
    sel_cat = [c for c in selected_features if c in CATEGORICAL_COLS]
    return sel_num, sel_cat

def build_preprocessor_selected(selected_features):
    sel_num, sel_cat = split_selected_features(selected_features)

    num_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
    cat_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", num_pipe, sel_num),
            ("cat", cat_pipe, sel_cat),
        ],
        remainder="drop",
        verbose_feature_names_out=False
    )
    return pre

def fit_transform_fold_selected(train_df, val_df, selected_features):
    pre = build_preprocessor_selected(selected_features)
    sel_num, sel_cat = split_selected_features(selected_features)

    Xtr_raw = train_df[sel_num + sel_cat]
    Xva_raw = val_df[sel_num + sel_cat]

    ytr = train_df[TARGET_COL].astype(int).to_numpy()
    yva = val_df[TARGET_COL].astype(int).to_numpy()

    Xtr = pre.fit_transform(Xtr_raw)
    Xva = pre.transform(Xva_raw)

    feat_names = pre.get_feature_names_out()
    return Xtr, ytr, Xva, yva, pre, feat_names

# sanity
tr_df, va_df = get_fold_split(df_labeled, fold_id=0)
Xtr_s, ytr_s, Xva_s, yva_s, pre_s, feat_s = fit_transform_fold_selected(tr_df, va_df, SELECTED_RAW_FEATURES)
print("Selected features transformed shape:", Xtr_s.shape, Xva_s.shape, "n_features:", len(feat_s))


Selected features transformed shape: (79132, 77) (19350, 77) n_features: 77


In [33]:
# =========================
# CELL 9 — Baseline LR (Spatial CV) using SELECTED_RAW_FEATURES
# =========================
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score, confusion_matrix

def run_spatial_cv_lr(df_labeled, selected_features, n_folds=5):
    rows = []
    cm_sum = np.zeros((4,4), dtype=int)

    for fid in range(n_folds):
        tr_df, va_df = get_fold_split(df_labeled, fold_id=fid)
        Xtr, ytr, Xva, yva, pre, feat_names = fit_transform_fold_selected(tr_df, va_df, selected_features)

        clf = LogisticRegression(
            max_iter=3000,
            n_jobs=-1,
            class_weight="balanced",
            multi_class="auto"
        )
        clf.fit(Xtr, ytr)
        pred = clf.predict(Xva)

        rows.append({
            "fold": fid,
            "acc": accuracy_score(yva, pred),
            "macro_f1": f1_score(yva, pred, average="macro"),
            "bal_acc": balanced_accuracy_score(yva, pred),
        })

        cm_sum += confusion_matrix(yva, pred, labels=[0,1,2,3])

    df_cv = pd.DataFrame(rows)
    print(df_cv)
    print("\nMean ± Std")
    display(df_cv[["acc","macro_f1","bal_acc"]].agg(["mean","std"]).round(4))

    cm_df = pd.DataFrame(cm_sum, index=[f"true_{i}" for i in range(4)], columns=[f"pred_{i}" for i in range(4)])
    print("\nSummed Confusion Matrix:")
    display(cm_df)

    return df_cv, cm_df

lr_cv, lr_cm = run_spatial_cv_lr(df_labeled, SELECTED_RAW_FEATURES, n_folds=N_FOLDS)


/Users/ashkan/miniconda3/envs/drought/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/ashkan/miniconda3/envs/drought/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/ashkan/miniconda3/envs/drought/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/ashkan/miniconda3/envs/drought/lib/python3.10/site-packag

   fold       acc  macro_f1   bal_acc
0     0  0.556693  0.325489  0.497146
1     1  0.317448  0.283991  0.356656
2     2  0.620106  0.283878  0.511119
3     3  0.448657  0.302085  0.308163
4     4  0.388534  0.312270  0.386582

Mean ± Std


,acc,macro_f1,bal_acc
mean,0.4663,0.3015,0.4119
std,0.1228,0.0181,0.0888



Summed Confusion Matrix:


,pred_0,pred_1,pred_2,pred_3
true_0,25592,5164,4143,772
true_1,8602,5613,7727,3712
true_2,8116,6832,9421,6286
true_3,148,801,1751,3802


In [34]:
# =========================
# CELL 10 — TabNet (Spatial CV) with MRMR-selected features
# =========================
import numpy as np
import pandas as pd
import torch

from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score, confusion_matrix

TABNET_PARAMS = dict(
    n_d=32,
    n_a=32,
    n_steps=4,
    gamma=1.5,
    lambda_sparse=1e-3,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-2),
    mask_type="entmax",
    scheduler_params={"step_size":10, "gamma":0.9},
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    seed=42,
    verbose=0
)

def run_spatial_cv_tabnet(df_labeled, selected_features, n_folds=5):
    rows = []
    cm_sum = np.zeros((4,4), dtype=int)

    for fid in range(n_folds):
        print(f"\n--- Fold {fid} ---")

        tr_df, va_df = get_fold_split(df_labeled, fold_id=fid)
        Xtr, ytr, Xva, yva, pre, feat_names = fit_transform_fold_selected(
            tr_df, va_df, selected_features
        )

        model = TabNetClassifier(**TABNET_PARAMS)

        model.fit(
            X_train=Xtr,
            y_train=ytr,
            eval_set=[(Xva, yva)],
            eval_name=["val"],
            eval_metric=["balanced_accuracy"],
            max_epochs=50,
            patience=10,
            batch_size=512,
            virtual_batch_size=128,
            num_workers=0,
            drop_last=False
        )

        pred = model.predict(Xva)

        rows.append({
            "fold": fid,
            "acc": accuracy_score(yva, pred),
            "macro_f1": f1_score(yva, pred, average="macro"),
            "bal_acc": balanced_accuracy_score(yva, pred),
        })

        cm_sum += confusion_matrix(yva, pred, labels=[0,1,2,3])

    df_cv = pd.DataFrame(rows)
    print("\nTabNet CV results:")
    display(df_cv)

    print("\nMean ± Std")
    display(df_cv[["acc","macro_f1","bal_acc"]].agg(["mean","std"]).round(4))

    cm_df = pd.DataFrame(
        cm_sum,
        index=[f"true_{i}" for i in range(4)],
        columns=[f"pred_{i}" for i in range(4)]
    )

    print("\nSummed Confusion Matrix:")
    display(cm_df)

    return df_cv, cm_df

tabnet_cv, tabnet_cm = run_spatial_cv_tabnet(
    df_labeled,
    SELECTED_RAW_FEATURES,
    n_folds=N_FOLDS
)



--- Fold 0 ---

Early stopping occurred at epoch 12 with best_epoch = 2 and best_val_balanced_accuracy = 0.59326


/Users/ashkan/miniconda3/envs/drought/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



--- Fold 1 ---

Early stopping occurred at epoch 11 with best_epoch = 1 and best_val_balanced_accuracy = 0.4721


/Users/ashkan/miniconda3/envs/drought/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



--- Fold 2 ---

Early stopping occurred at epoch 11 with best_epoch = 1 and best_val_balanced_accuracy = 0.47771


...

,fold,acc,macro_f1,bal_acc
0,0,0.625633,0.422084,0.593262
1,1,0.425828,0.417627,0.472101
2,2,0.544394,0.300226,0.477708
3,3,0.554487,0.385624,0.398233
4,4,0.495799,0.435759,0.471962



Mean ± Std


,acc,macro_f1,bal_acc
mean,0.5292,0.3923,0.4827
std,0.0741,0.0546,0.0700



Summed Confusion Matrix:


,pred_0,pred_1,pred_2,pred_3
true_0,26711,3921,4833,206
true_1,6763,9692,7829,1370
true_2,5705,8121,13008,3821
true_3,15,861,3257,2369
